In [ ]:
%matplotlib inline
import os.path as op

import gensim
from keras.applications import vgg19
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from nltk.corpus import stopwords
import numpy as np
from PIL import Image
from pycocotools.coco import COCO

from image_processing import process_image
from retrieval_pipelines import absolute_coco_path, tag_to_image_search, image_to_tag_search, most_common_tags
from text_processing import sentence2vec, create_caption_dataframe
from tools import intersect_sort
from vgg import compute_nn_features
from word2vec import compute_textual_features

In [ ]:
data_dir = "dataset"
data_type = "train2014"
ann_file = op.join(
    data_dir, "annotations", "instances_{0}.json".format(data_type))
coco = COCO(ann_file)

In [ ]:
W_visual = np.load("W1.npy")
W_text = np.load("W2.npy")
V = np.load("V.npy")
T = np.load("T.npy")
img_ids = np.load("visual_img_ids.npy")
W_text = W_text.T
W_visual = W_visual.T

In [ ]:
tag = "sport"

In [ ]:
# print('\nLoading word2vec model ...')
# path = op.join('models', 'GoogleNews-vectors-negative300.bin')
# model = gensim.models.Word2Vec.load_word2vec_format(path, binary=True)

In [ ]:
# tag_features = sentence2vec(tag, model)
# del model
tag_features = np.load("{0}_features.npy".format(tag))

In [ ]:
# np.save("{0}_features".format(tag), tag_features)

In [ ]:
database_images = W_visual.dot(V.T)
database_images = database_images.T

In [ ]:
retrieved_img_ids = tag_to_image_search(tag_features, W_text, database_images, img_ids, coco, n_images=10)
img_paths = [absolute_coco_path(int(img_id), coco) for img_id in retrieved_img_ids]

In [ ]:
for im_idx, img_path in enumerate(img_paths):
    plt.figure(im_idx)
    img = mpimg.imread(img_path)
    imgplot = plt.imshow(img)

In [ ]:
# df_caption = create_caption_dataframe()
# T = compute_textual_features(df_caption)
# textual_img_ids = df_caption.index.values
# Take only the features corresponding to common ids and sort by id
# V, visual_img_ids, T, textual_img_ids = intersect_sort(V, img_ids, T, textual_img_ids)
# np.save("T", T)

In [ ]:
database_captions = W_text.dot(T.T)
database_captions = database_captions.T

In [ ]:
image_name = "boat.jpg"
image_path = op.join(data_dir, "test", image_name)

In [ ]:
plt.figure()
plt.imshow(mpimg.imread(image_path))

In [ ]:
# img_mat = process_image(image_path)

In [ ]:
# net = vgg19.VGG19()
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# net.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
# visual_features = compute_nn_features([img_mat], net)

In [ ]:
# np.save("{0}_visual_features".format(image_name.split(".")[0]), visual_features)

In [ ]:
visual_features = np.load("{0}_visual_features.npy".format(image_name.split(".")[0]))
visual_features = np.reshape(visual_features, (W_visual.shape[1]))

In [ ]:
n_tags = 4
annotations = image_to_tag_search(visual_features.T, W_visual, database_captions, img_ids, coco, n_tags=n_tags, expanding_factor=4)

In [ ]:
stops = set(stopwords.words('english'))

In [ ]:
most_common_tags(annotations, n_tags, stops)